# Práctica 1: Setup y Benchmark de I/O (CSV vs Parquet)

## Objetivos
1.  Configurar el entorno con `uv`.
2.  Generar un dataset sintético.
3.  Comparar tiempos de lectura y tamaños entre CSV y Parquet.
4.  **BONUS A**: Column Pruning.
5.  **BONUS B**: Saturación de RAM y gestión de memoria.

## 1. Setup inicial
Asegúrate de haber instalado las dependencias:
```bash
uv sync
```

In [ ]:
import pandas as pd
import time
import os
import sys
import psutil
import logging
import matplotlib.pyplot as plt
from pathlib import Path

# Configurar logging básico para el notebook
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('notebook')

# Añadimos el root del proyecto al path
sys.path.append('..')
from src.utils import generate_synthetic_data, measure_rss_mb

## 2. Generación de Datos
Generamos dataset sintético.

In [ ]:
DATA_DIR = Path('../data/raw')
CSV_PATH = DATA_DIR / 'synthetic.csv'
N_ROWS = 1_000_000

generate_synthetic_data(CSV_PATH, n_rows=N_ROWS)

size_mb = CSV_PATH.stat().st_size / (1024 * 1024)
logger.info(f"CSV Size: {size_mb:.2f} MB")

## 3. Benchmarks (Resumen)
Para ahorrar espacio, convertimos a Parquet y hacemos una lectura rápida para comprobar que todo funciona.

In [ ]:
PARQUET_PATH = CSV_PATH.with_suffix('.parquet')

# CSV -> Parquet
df = pd.read_csv(CSV_PATH)
df.to_parquet(PARQUET_PATH)
logger.info("Conversión completada.")

# Comparativa rápida
pq_size_mb = PARQUET_PATH.stat().st_size / (1024 * 1024)
logger.info(f"Size CSV: {size_mb:.2f} MB vs Parquet: {pq_size_mb:.2f} MB")

## 4. OOM (Out Of Memory) Demo
Vamos a intentar saturar la RAM. **¡Cuidado!** Esto puede ralentizar tu máquina.
Crearemos una lista gigante de enteros hasta que salte `MemoryError` o el kernel muera.

### Monitor de RAM
Usaremos `psutil` para ver cómo sube la memoria.

In [ ]:
def oom_demonstration():
    long_list = []
    memory_usage = []
    
    logger.warning("Iniciando prueba de estrés de memoria...")
    try:
        # Intentamos alocar chunks grandes
        for i in range(1000):
            # Appending a big list of integers each time
            # 10M integers ~ 80MB per chunk roughly in python list overhead
            chunk = list(range(1_000_000))
            long_list.append(chunk)
            
            # Monitor
            mem = measure_rss_mb()
            memory_usage.append(mem)
            
            if i % 10 == 0:
                logger.info(f"Iter {i}: {mem:.2f} MB RSS")
                
            # Safety break (opcional, quitar si quieres crash real)
            if mem > 8 * 1024:  # Stop at 8GB to act safe
                logger.error("¡Límite de seguridad alcanzado (8GB)! Parando antes de bloquear el sistema.")
                break
                
    except MemoryError:
        logger.critical("¡MemoryError capturado! El proceso se quedó sin RAM.")
    except Exception as e:
        logger.critical(f"El proceso murió o falló: {e}")
    finally:
        # Plot results
        plt.figure(figsize=(10, 5))
        plt.plot(memory_usage, label='RSS Memory (MB)')
        plt.title('Consumo de RAM durante OOM Demo')
        plt.xlabel('Chunks')
        plt.ylabel('MB')
        plt.grid(True)
        plt.legend()
        plt.show()
        
        # Clean up immediately
        del long_list
        import gc
        gc.collect()
        logger.info("Memoria liberada.")

oom_demonstration()